# Lab2 - Hate speech classification - Alicia, Arik, Jonas


# Get existing data

To train the model and get the required files, please first run hate_speech_classification_baseline.ipynb

In [3]:
import os.path
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

if os.path.exists('trained_classifier.pkl'):
    with open('trained_classifier.pkl', 'rb') as fid:
        clf = pickle.load(fid)
else:
    print("No trained classifier found")
    exit()

X = pickle.load(open('X.pkl', 'rb'))
tfidfizer = TfidfVectorizer(max_features=1000)
tfidfizer.fit_transform(X)

<159571x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2842337 stored elements in Compressed Sparse Row format>

# Get Youtube comments

Getting the 100 latest comments from the current top 50 trending videos.

Replace GOOGLE_API_KEY with your Google cloud api key. You can get a key from https://console.developers.google.com/apis/credentials

You also have to activate the Youtube Data API v3 on https://console.developers.google.com/apis/library/youtube.googleapis.com

In [22]:
import csv
import requests

maxResults = '100'
max_trending_videos = '50'
apiKey = 'GOOGLE_API_KEY'
comments = {}
all_comments = []


def get_comments(video_id):
    endpoint = 'https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults={0}&videoId={1}&fields=items(id%2Csnippet%2FtopLevelComment%2Fsnippet%2FtextDisplay)&key={2}'.format(maxResults, video_id, apiKey)
    r = requests.get(endpoint)
    comments = []
    if r.status_code == 200:
        for item in r.json().get('items'):
            comment = {}
            comment['comment_id'] = item.get('id')
            comment['comment'] = item.get('snippet').get('topLevelComment').get('snippet').get('textDisplay')
            comments.append(comment)
    else:
        print(r.json())
    return comments
                   
                
def get_trending_videos():
    endpoint = 'https://www.googleapis.com/youtube/v3/videos?part=snippet&chart=mostPopular&fields=items(id%2Csnippet%2Ftitle%2Csnippet%2FchannelId%2Csnippet%2FchannelTitle)&maxResults={0}&key={1}'.format(max_trending_videos, apiKey)
    r = requests.get(endpoint)
    items = r.json().get('items')
    
    for item in items:
        video_id = item.get('id')
        title = item.get('snippet').get('title')
        comments = get_comments(video_id)
        if comments:
            for comment in comments:
                comment_tuple = (video_id, title, comment['comment_id'], comment['comment'])
                all_comments.append(comment_tuple)
            
    
def write_csv():
    with open('comments.csv', mode='w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['ID', 'VIDEOID', 'COMMENT', 'VIDEONAME'])
        
        for comment in all_comments:
            video_id = comment[0]
            video_name = comment[1]
            id = comment[2]
            comment = comment[3].replace('<br />', ' ').replace('<b>', ' ').encode('ascii', 'ignore').decode('ascii')
            comments_writer.writerow([id, video_id, comment, video_name])


print('Getting comments from Youtube...')
get_trending_videos()
write_csv()
print('finished writing csv')


Getting comments from Youtube...
finished writing csv


In [23]:
import pandas as pd

df = pd.read_csv('comments.csv')
df.columns = ['ID', 'VIDEOID', 'COMMENT', 'VIDEONAME']
    
video_ids = df['VIDEOID'].drop_duplicates()

comments = df['COMMENT']

# Test on new data

In [24]:
result = {}

for videoId in video_ids:
    result[videoId] = {'hate_comment_count': 0, 'total_comment_count': 0, 'video_name': ''}

In [25]:
tf_idf_matrix_comments = tfidfizer.transform(comments.astype('U'))
comments_result = clf.predict(tf_idf_matrix_comments)

In [26]:
from IPython.display import display, Markdown

previous_video_id = ''
cnt = 0
for index, row in df.iterrows():
    cnt += 1
    is_hate_speech = comments_result[index]
    current_video_id = row['VIDEOID']
    
    if(current_video_id != previous_video_id):
        title = '### '+ row['VIDEONAME'] + ' (' + row['VIDEOID'] + ') hate speech:'
        display(Markdown(title))
    
    previous_video_id = current_video_id;
    
    result_by_video_id = result.get(current_video_id);
    
    if result_by_video_id is not None:

        if is_hate_speech == 1:
            item = '* ' + row['COMMENT']
            result_by_video_id['hate_comment_count'] += 1
            display(Markdown(item))

        result_by_video_id['total_comment_count'] += 1
        
        result_by_video_id['video_name'] = row['VIDEONAME']

### Taylor Swift - ME! (feat. Brendon Urie of Panic! At The Disco) (FuXNumBwDOM) hate speech:

* Love u from VetNam

* I thought the old Taylor was dead? lol

* <a href="https://www.youtube.com/watch?v=FuXNumBwDOM&amp;t=2m17s">2:17</a> so this is what the inside of Brendon looks like...WOW

*  Record :</b> most viewed youtube video in the first 24 hours

...

* i hope the movie looks better no hate its just clear that they are using cgi in this one if the trailer looks like the movie in this one then 2014 was better animated im just sayin

* Where the fuck ultraman is when we need them?

* Love milli brown

* I love Godzilla movie

* Producer: Cue stranger than things actress. Producer: Cue all the other shit. Producer: Action.   Audience: Fuck it. Looks like 15$ more than i want to spend.   Prediction: No plot. No story, all graphics. Waste your money.

In [12]:
most_hate_comment_count_video_id= ''
most_hate_comment_count = 0;

for key, value in result.items():
    if (value.get('hate_comment_count') > most_hate_comment_count):
        most_hate_comment_count = value.get('hate_comment_count')
        most_hate_comment_count_video_id = key
        
direct_url = 'https://www.youtube.com/watch?v=' + most_hate_comment_count_video_id

In [13]:
print('Most hated trending video on youtube: ')
print(direct_url)

Most hated trending video on youtube: 
https://www.youtube.com/watch?v=c8Mljz89-v4


In [14]:
from IPython.display import HTML
embed_url = 'https://www.youtube.com/embed/' + most_hate_comment_count_video_id
HTML('<iframe width="560" height="315" src="{0}" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'.format(embed_url))